# Privacera Shield protecting Python OpenAI Application
This notebook shows how to use Privacera Shield Library with a Python application that is using OpenAI API. To run this notebook you will need the following,


1.  Sign up for a free account at [Privacera AI Governance (PAIG)](https://privacera.ai). This is simple, all you need is your email address.
2.  OpenAI API Key. This will allow you to create your first OpenAI application governed by Privacera AI Governance. Please visit your [OpenAI account](https://platform.openai.com) to get this key.

# Step 1 - Install the Python packages
This step will take about a minute to complete.

In [ ]:
!pip install -qqq openai==1.25.1 privacera_shield==1.1.12

# Step 2 - Upload Privacera AI application configuration file into Collab
1. Log into your account in PAIG.
2. To create a new application, go to `Application > AI Applications` and click the `CREATE APPLICATION` button on the right top. This will open a dialog box where you can enter the details of the application.
3. Navigate to `Application -> AI Applications` and select the application you want to generate the api key. In the `API KEYS` tab, click the `GENERATE API KEY` button in the top-right corner to generate an API key. Provide a `Name` and `Description`, along with a `Expiry`, or select the `Max Validity (1 year)` checkbox to set default expiry.

Note: Once the Api Key is generated, it will not be displayed again. Ensure you copy and securely store it immediately after generation.

# Step 3 - Configure your OpenAI API Key into the environment.
Enter your OpenAI API key in the pop-up. This key is stored in the environment of the notebook and not uploaded to PAIG.

Hit enter after you have pasted the key.

In [ ]:
import os
from getpass import getpass

if os.environ.get("OPENAI_API_KEY") is None:
    openai_api_key = getpass("🔑 Enter your OpenAI API key and hit Enter:")
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Step 4 - Import all the required packages

In [ ]:
import os
import uuid

from privacera_shield import client as privacera_shield_client
from privacera_shield.exception import AccessControlException
from privacera_shield.model import ConversationType

from openai import OpenAI

# utility function to wrap the output
def wrap_text(text, width=80):
    words = text.split()
    character_count = 0
    for word in words:
        if character_count + len(word) + 1 > width:  # Check if adding the word would exceed the width
            print("\n", end="")  # Start a new line
            character_count = 0  # Reset the character count for the new line
        print(word, end=" ")  # Print the word followed by a space
        character_count += len(word) + 1  # Update the character count
    print()

# Step 5 - Initialize the Privacera Shield library

To initialize the Privacera Shield library in your AI application, set the PAIG_API_KEY either as an environment variable or directly in the code. Choose one of the following methods:

1. Set the PAIG_API_KEY as an environment variable.


<p style="margin-left: 40px;">
  <span style="font-size: 20px;">•</span> Enter the PAIG API key obtained from the PAIG portal into the pop-up and press Enter to confirm.
</p>

In [ ]:
import os
from getpass import getpass

if os.environ.get("PAIG_API_KEY") is None:
    paig_api_key = getpass("🔑 Enter your PAIG API key and hit Enter:")
    os.environ["PAIG_API_KEY"] = paig_api_key

# Initialize Privacera Shield
privacera_shield_client.setup(frameworks=[])

<p style="text-align: center;">
  <span style="font-size: 30px; font-weight: bold;">OR</span>
</p>


2. Pass the API Key in Code
    
    If exporting the PAIG_API_KEY as an environment variable is not preferred, pass it directly as an argument.
    Make Sure to replace API_KEY with the actual apikey generated from paig portal.

In [ ]:
# Set the PAIG_API_KEY environment variable or set it here
privacera_shield_client.setup(frameworks=[], application_config_api_key="<API_KEY\>")

#### Note: The frameworks parameter is kept empty in both methods.
> **Tip:** If you get an error in this step, it could be because you have already run this step once. You can ignore the error and continue. You can also try restarting the Kernel and start from the beginning with the proper JSON file and OpenAI API key.

# Step 6 - Initialize OpenAI based LLM object in LangChain
In this step, we are creating an LLM object for the OpenAI LLM and a prompt template.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")  # (1)

llm = OpenAI()

# Step 7 - Run the application
1. In this step, we are going to run the application with the prompt asked by a user named `testuser`. 
2. Note how we are passing the username by creating a Privacera Shield context object. 
3. Within the context object, application developer makes an API calls to PAIG using check_access() API - one call for the Prompt text and other for Response text. 
4. The PAIG service scans both the prompt and response text and runs security policies. 
5.If the policies decide that the access is denied then an AccessControlException is thrown.

> **Note:** Here username used is `testuser` which is an external user. So the policies applied will be as per the public. For applying userspecific properties, create and use the user from PAIG portal `Account > User Management > User`.

In [ ]:
# Generate a random UUID which will be used to bind a prompt with a reply
privacera_thread_id = str(uuid.uuid4())

# Let's assume the user is "testuser"
user = "testuser"

try:
    with privacera_shield_client.create_shield_context(username=user):

        # Create a user prompt
        prompt_text = "Who was the first President of USA and where did they live?"
        wrap_text(f"User Prompt: {prompt_text}\n")

        # Validate prompt with Privacera Shield
        updated_prompt_text = privacera_shield_client.check_access(
          text=prompt_text,
          conversation_type=ConversationType.PROMPT,
          thread_id=privacera_thread_id
        )
        updated_prompt_text = updated_prompt_text[0].response_text
        wrap_text(f"User Prompt (After Privacera Shield): {updated_prompt_text}")

        # Call LLM with updated prompt text
        PROMPT = f"""Use the following pieces of context to answer the question at the end.
        {updated_prompt_text}
        ANSWER:
        """

        response = llm.chat.completions.create(model="gpt-4",
                                messages=[{"role": "user", "content": PROMPT}],
                                temperature=0)

        llm_response = response.choices[0].message.content
        wrap_text(f"LLM Response: {llm_response}")

        # Validate LLM response with Privacera Shield
        updated_reply_text = privacera_shield_client.check_access(
            text=llm_response,
            conversation_type=ConversationType.REPLY,
            thread_id=privacera_thread_id)
        updated_reply_text = updated_reply_text[0].response_text

        wrap_text(f"LLM Response (After Privacera Shield): {updated_reply_text}")

except AccessControlException as e:
    # If access is denied, then this exception will be thrown. You can handle it accordingly.
    print(f"AccessControlException: {e}")

# Step 8 - Review the access audits in PAIG portal under Security menu option
1. Now you can log in to the PAIG portal and check under `Security > Access Audits` section. You will see the audit record for the above run of your LangChain application.
![title](img/Access_Audits.png)
2. You can click on the eye icon and see the details of the prompts sent by the application to the LLM and the responses coming from the LLM.
![title](img/Access_Audits_2.png)
3. The default policy in PAIG for the application monitors the flow and tags the contents of the prompt and response.

# Step 9 - Modify the policy of your AI application
1. Log into the PAIG portal and navigate to your application under Application section. 
2. Click on the Permissions tab and enable the 'CCPA Personal Information Redaction' policy by clicking on the toggle in the status column. This will start redacting the sensitive data in the Prompt and Reply.
![title](img/AI_App_Policy.png)
3. Wait for few seconds and re-run Step 7 above. The reply will have the Location and Person name redacted, that is, replaced by `<<LOCATION>>` and `<<PERSON>>`. You can re-run Step 7 a few more times if the above is not redaction is not reflected.
4. Then check the access audits and you will see the original response from the LLM and how Privacera AI redacted the sensitive information out of it.
5. You can also modify the policy by clicking on the pencil icon and change the Prompt to Allow while keeping Reply to Redact.
![title](img/AI_App_Policy_2.png)
6. Wait for few seconds and re-run Step 7. In the access audits, you will see that the prompt is unchanged while the response is redacted.
7. You can change the question in Step 7 and also try changing the policy to see the effect.

# Next steps
Continue your journey with PAIG by following these sections in the documentation link in the PAIG portal,
- [Integrating with Langchain application](https://na.privacera.ai/docs/integration/langchain.html) 
- [Integrating with Python application](https://na.privacera.ai/docs/integration/python-applications.html)
- [Try the Privacera SecureChat](https://na.privacera.ai/docs/integration/securechat.html)
- [PAIG User Guide](https://na.privacera.ai/docs/user-guide/index.html)